In [ ]:
from pathlib import Path
import pandas as pd
file_path = Path("data/dataset_126.csv")

try:

    df_126 = pd.read_csv(file_path, encoding='utf-8')
except pd.errors.ParserError:
    try:
    
        df_126 = pd.read_csv(file_path, encoding='utf-8', sep=';')
    except pd.errors.ParserError:
    
        df_126 = pd.read_csv(file_path, encoding='utf-8', header=None)

df_126.head(5)

,case_id,case_hash,region_id,region_name,case_type,instance,court_name,court_id,case_number,entry_date,...,end_date,considered_by,cui,is_text,meta_link,text_link,connected_links,db_add_date,db_update_date,text
0,259727021925,3c78efd765,45,Курганская область,CRIMINAL,FIRST,Курганский городской суд Курганской области,45RS0026,1-88/2020 (1-1518/2019;),2019-10-10,...,2020-03-11,Рассмотрено единолично судьей,45RS0026-01-2019-011884-16,True,http://kurgansky--krg.sudrf.ru/modules.php?nam...,NaN,NaN,2024-08-07,2024-08-07,Дело №\nП Р И Г О В О Р\nИменем Российской Фед...
1,1026059089394,eee5e335f2,72,Тюменская область,CRIMINAL,FIRST,Ялуторовский районный суд Тюменской области,72RS0028,1-11/2020 (1-199/2019;),2019-11-18,...,2020-05-14,Рассмотрено единолично судьей,72RS0028-01-2019-001364-46,False,http://yalutorovsky--tum.sudrf.ru/modules.php?...,NaN,NaN,2024-08-07,2024-08-07,NaN
2,1043013970412,f2d87a29ec,47,Ленинградская область,CRIMINAL,FIRST,Ленинградский областной суд,47OS0000,2-5/2020 (2-22/2019;),2019-06-03,...,2020-06-23,Рассмотрено с участием присяжных заседателей,47OS0000-01-2019-000184-94,True,http://oblsud--lo.sudrf.ru/modules.php?name=su...,http://oblsud--lo.sudrf.ru/modules.php?name=su...,NaN,2024-08-07,2024-08-07,Дело № 2-5/2020\nП Р И Г О В О Р\nИменем Росси...
3,564224541856,835e6744a0,44,Костромская область,CRIMINAL,FIRST,Вохомский районный суд Костромской области,44RS0015,1-8/2020 (1-73/2019;),2019-12-30,...,2020-02-18,Рассмотрено единолично судьей,44RS0015-01-2019-000357-91,False,http://vohomsky--kst.sudrf.ru/modules.php?name...,NaN,NaN,2024-08-07,2024-08-07,NaN
4,489656147605,7201c7de95,70,Томская область,CRIMINAL,FIRST,Октябрьский районный суд г.Томска,70RS0003,1-6/2020 (1-249/2019;),2019-03-11,...,8888-01-01,Рассмотрено единолично судьей,70RS0003-01-2019-001389-85,False,http://oktiabrsky--tms.sudrf.ru/modules.php?na...,NaN,NaN,2024-08-07,2024-08-07,NaN


In [16]:
import pandas as pd
import re

df_126['accused'] = df_126['accused'].str.replace(r'\s+', ' ', regex=True).str.strip()
df_126['accused'] = df_126['accused'].str.replace(r'[–—]', '-', regex=True)

# Регулярное выражение для статьи УК РФ
article_pattern = r'(ст\.?\s*126(?:\s*ч\.?\s*[1-3])?)'

# Функция для извлечения уникальных статей в блоках с "<Информация скрыта>"
def extract_hidden_articles(text):
    results = []
    blocks = re.split(r'(<Информация скрыта>)', text)
    
    for i in range(len(blocks)):
        if blocks[i] == "<Информация скрыта>":
            if i + 1 < len(blocks):
                articles = set(re.findall(article_pattern, blocks[i + 1], flags=re.IGNORECASE))
                results.append(len(articles))
    
    return sum(results)

# Функция для извлечения уникальных статей по каждому доступному ФИО
def extract_fio_with_articles(text):
    fio_pattern = (
        r'([А-ЯЁA-Z]+(?:\s[А-ЯЁA-Z]+){2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z][а-яёa-z]+){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z]\.){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s(?:уулу|оглы|кызы))\s[А-ЯЁA-Z][а-яёa-z]+'
        r'|<Информация скрыта>)'
    )
    fio_matches = re.findall(fio_pattern, text)
    results = []
    start_index = 0

    for i, fio in enumerate(fio_matches):
        fio = fio.strip()
        if fio == '<Информация скрыта>':
            continue

        fio_index = text.find(fio, start_index)
        if i + 1 < len(fio_matches):
            next_fio = fio_matches[i + 1]
            next_fio_index = text.find(next_fio, fio_index)
            text_after_fio = text[fio_index + len(fio): next_fio_index]
        else:
            text_after_fio = text[fio_index + len(fio):]
        
        articles = set(re.findall(article_pattern, text_after_fio, flags=re.IGNORECASE))
        results.append(len(articles))
        start_index = fio_index + len(fio)
    
    return sum(results)

# Применяем функции
df_126['hidden_articles_count'] = df_126['accused'].apply(extract_hidden_articles)
df_126['unique_articles_count'] = df_126['accused'].apply(extract_fio_with_articles)

# Считаем сумму двух колонок для доступных ФИО и "<Информация скрыта>"
df_126['total_articles_count'] = df_126['hidden_articles_count'] + df_126['unique_articles_count']

df_126[['case_number', 'hidden_articles_count', 'unique_articles_count', 'total_articles_count']].head(20)

total_articles_overall = df_126['total_articles_count'].sum()
print(f"Общее количество статей по всему датасету: {total_articles_overall}")


Общее количество статей по всему датасету: 5898


In [ ]:
import pandas as pd
file_path = Path("data/dataset_127.csv")

try:

    df_127 = pd.read_csv(file_path, encoding='utf-8')
except pd.errors.ParserError:
    try:
    
        df_127 = pd.read_csv(file_path, encoding='utf-8', sep=';')
    except pd.errors.ParserError:
    
        df_127 = pd.read_csv(file_path, encoding='utf-8', header=None)

df_127.head(5)

,case_id,case_hash,region_id,region_name,case_type,instance,court_name,court_id,case_number,entry_date,...,end_date,considered_by,cui,is_text,meta_link,text_link,connected_links,db_add_date,db_update_date,text
0,5665476727,0151b05077,2,Республика Башкортостан,CRIMINAL,FIRST,Нефтекамский городской суд Республики Башкорто...,03RS0013,1-394/2015,2015-10-05,...,2015-11-24,Рассмотрено единолично судьей,03RS0013-01-2015-005046-89,True,http://neftekamsky--bkr.sudrf.ru/modules.php?n...,NaN,NaN,2024-08-06,2024-08-06,Дело № 1-394/2015\nП Р И Г О В О Р\nИменем Рос...
1,30077325714,0700bf9192,2,Республика Башкортостан,CRIMINAL,FIRST,Белебеевский городской суд Республики Башкорто...,03RS0009,1-186/2015,2015-08-10,...,2015-12-30,Рассмотрено единолично судьей,03RS0009-01-2014-003290-24,True,http://belebeevsky--bkr.sudrf.ru/modules.php?n...,http://belebeevsky--bkr.sudrf.ru/modules.php?n...,NaN,2024-08-06,2024-08-06,Дело № 1-186/2015\nПРИГОВОР\nИменем Российской...
2,996367930263,e7fc281397,86,Ханты-Мансийский автономный округ — Югра,CRIMINAL,FIRST,Нефтеюганский районный суд Ханты-Мансийского а...,86RS0007,1-15/2015 (1-154/2014;),2014-02-03,...,2016-05-18,Рассмотрено единолично судьей,86RS0007-01-2014-005160-11,False,http://uganskray--hmao.sudrf.ru/modules.php?na...,NaN,NaN,2024-08-06,2024-08-06,NaN
3,225970250940,349ce064bc,48,Липецкая область,CRIMINAL,FIRST,Липецкий областной суд,48OS0000,2-4/2015,2015-07-24,...,2015-12-18,Рассмотрено единолично судьей,48OS0000-01-2015-000046-31,False,http://oblsud--lpk.sudrf.ru/modules.php?name=s...,NaN,NaN,2024-08-06,2024-08-06,NaN
4,1066032581988,f8347e4d64,50,Московская область,CRIMINAL,FIRST,Одинцовский городской суд Московской области,50RS0031,1-504/2015,2015-06-22,...,2015-10-31,Рассмотрено единолично судьей,50RS0031-01-2015-019306-91,True,http://odintsovo--mo.sudrf.ru/modules.php?name...,http://odintsovo--mo.sudrf.ru/modules.php?name...,NaN,2024-08-06,2024-08-06,П О С Т А Н О В Л Е Н И Е\nо прекращении уголо...


In [20]:
import pandas as pd
import re

df_127['accused'] = df_127['accused'].str.replace(r'\s+', ' ', regex=True).str.strip()
df_127['accused'] = df_127['accused'].str.replace(r'[–—]', '-', regex=True)

# Регулярное выражение для статьи УК РФ
article_pattern = r'(ст\.?\s*127(?:\s*ч\.?\s*[1-3])?)'

# Функция для извлечения уникальных статей в блоках с "<Информация скрыта>"
def extract_hidden_articles(text):
    results = []
    blocks = re.split(r'(<Информация скрыта>)', text)
    
    for i in range(len(blocks)):
        if blocks[i] == "<Информация скрыта>":
            if i + 1 < len(blocks):
                articles = set(re.findall(article_pattern, blocks[i + 1], flags=re.IGNORECASE))
                results.append(len(articles))
    
    return sum(results)

# Функция для извлечения уникальных статей по каждому доступному ФИО
def extract_fio_with_articles(text):
    fio_pattern = (
        r'([А-ЯЁA-Z]+(?:\s[А-ЯЁA-Z]+){2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z][а-яёa-z]+){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z]\.){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s(?:уулу|оглы|кызы))\s[А-ЯЁA-Z][а-яёa-z]+'
        r'|<Информация скрыта>)'
    )
    fio_matches = re.findall(fio_pattern, text)
    results = []
    start_index = 0

    for i, fio in enumerate(fio_matches):
        fio = fio.strip()
        if fio == '<Информация скрыта>':
            continue

        fio_index = text.find(fio, start_index)
        if i + 1 < len(fio_matches):
            next_fio = fio_matches[i + 1]
            next_fio_index = text.find(next_fio, fio_index)
            text_after_fio = text[fio_index + len(fio): next_fio_index]
        else:
            text_after_fio = text[fio_index + len(fio):]
        
        articles = set(re.findall(article_pattern, text_after_fio, flags=re.IGNORECASE))
        results.append(len(articles))
        start_index = fio_index + len(fio)
    
    return sum(results)

# Применяем функции
df_127['hidden_articles_count'] = df_127['accused'].apply(extract_hidden_articles)
df_127['unique_articles_count'] = df_127['accused'].apply(extract_fio_with_articles)

# Считаем сумму двух колонок для доступных ФИО и "<Информация скрыта>"
df_127['total_articles_count'] = df_127['hidden_articles_count'] + df_127['unique_articles_count']

df_127[['case_number', 'hidden_articles_count', 'unique_articles_count', 'total_articles_count']].head(20)

total_articles_overall = df_127['total_articles_count'].sum()
print(f"Общее количество статей по всему датасету: {total_articles_overall}")


Общее количество статей по всему датасету: 3357


In [ ]:
import pandas as pd
file_path = Path("data/dataset_240.csv")

try:

    df_240 = pd.read_csv(file_path, encoding='utf-8')
except pd.errors.ParserError:
    try:
    
        df_240 = pd.read_csv(file_path, encoding='utf-8', sep=';')
    except pd.errors.ParserError:
    
        df_240 = pd.read_csv(file_path, encoding='utf-8', header=None)

df_240.head(5)

,case_id,case_hash,region_id,region_name,case_type,instance,court_name,court_id,case_number,entry_date,...,end_date,considered_by,cui,is_text,meta_link,text_link,connected_links,db_add_date,db_update_date,text
0,759465530388,b0d3ac5414,24,Красноярский край,CRIMINAL,FIRST,Советский районный суд г.Красноярска,24RS0048,1-854/2018,2018-08-03,...,2018-10-09,Рассмотрено единолично судьей,24RS0048-01-2018-009526-31,False,http://sovet--krk.sudrf.ru/modules.php?name=su...,NaN,NaN,2024-08-06,2024-08-06,NaN
1,117770368922,1b6ba9379a,64,Саратовская область,CRIMINAL,FIRST,Октябрьский районный суд г. Саратова,64RS0047,1-77/2018,2018-05-11,...,2019-03-07,Рассмотрено единолично судьей,NaN,False,http://oktyabrsky--sar.sudrf.ru/modules.php?na...,NaN,NaN,2024-08-06,2024-08-06,NaN
2,214435330884,31ed577b44,61,Ростовская область,CRIMINAL,FIRST,Краснодарский гарнизонный военный суд,61GV0004,1-36/2018,2018-07-31,...,2018-11-22,Рассмотрено единолично судьей,61GV0004-01-2017-000073-53,False,http://gvs--krd.sudrf.ru/modules.php?name=sud_...,NaN,NaN,2024-08-06,2024-08-06,NaN
3,811129576574,bcdb171c7e,63,Самарская область,CRIMINAL,FIRST,Октябрьский районный суд г. Самары,63RS0040,1-192/2018,2018-05-24,...,2018-09-24,NaN,NaN,False,http://oktyabrsky--sam.sudrf.ru/modules.php?na...,NaN,NaN,2024-08-06,2024-08-06,NaN
4,400222903773,5d2f24dddd,47,Ленинградская область,CRIMINAL,FIRST,Гатчинский городской суд Ленинградской области,47RS0006,1-176/2024 (1-688/2023;),2023-12-28,...,NaN,NaN,NaN,False,http://gatchinsky--lo.sudrf.ru/modules.php?nam...,NaN,NaN,2024-08-06,2024-08-06,NaN


In [22]:
import pandas as pd
import re

df_240['accused'] = df_240['accused'].str.replace(r'\s+', ' ', regex=True).str.strip()
df_240['accused'] = df_240['accused'].str.replace(r'[–—]', '-', regex=True)

# Регулярное выражение для статьи УК РФ
article_pattern = r'(ст\.?\s*240(?:\s*ч\.?\s*[1-3])?)'

# Функция для извлечения уникальных статей в блоках с "<Информация скрыта>"
def extract_hidden_articles(text):
    results = []
    blocks = re.split(r'(<Информация скрыта>)', text)
    
    for i in range(len(blocks)):
        if blocks[i] == "<Информация скрыта>":
            if i + 1 < len(blocks):
                articles = set(re.findall(article_pattern, blocks[i + 1], flags=re.IGNORECASE))
                results.append(len(articles))
    
    return sum(results)

# Функция для извлечения уникальных статей по каждому доступному ФИО
def extract_fio_with_articles(text):
    fio_pattern = (
        r'([А-ЯЁA-Z]+(?:\s[А-ЯЁA-Z]+){2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z][а-яёa-z]+){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z]\.){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s(?:уулу|оглы|кызы))\s[А-ЯЁA-Z][а-яёa-z]+'
        r'|<Информация скрыта>)'
    )
    fio_matches = re.findall(fio_pattern, text)
    results = []
    start_index = 0

    for i, fio in enumerate(fio_matches):
        fio = fio.strip()
        if fio == '<Информация скрыта>':
            continue

        fio_index = text.find(fio, start_index)
        if i + 1 < len(fio_matches):
            next_fio = fio_matches[i + 1]
            next_fio_index = text.find(next_fio, fio_index)
            text_after_fio = text[fio_index + len(fio): next_fio_index]
        else:
            text_after_fio = text[fio_index + len(fio):]
        
        articles = set(re.findall(article_pattern, text_after_fio, flags=re.IGNORECASE))
        results.append(len(articles))
        start_index = fio_index + len(fio)
    
    return sum(results)

# Применяем функции
df_240['hidden_articles_count'] = df_240['accused'].apply(extract_hidden_articles)
df_240['unique_articles_count'] = df_240['accused'].apply(extract_fio_with_articles)

# Считаем сумму двух колонок для доступных ФИО и "<Информация скрыта>"
df_240['total_articles_count'] = df_240['hidden_articles_count'] + df_240['unique_articles_count']

df_240[['case_number', 'hidden_articles_count', 'unique_articles_count', 'total_articles_count']].head(20)

total_articles_overall = df_240['total_articles_count'].sum()
print(f"Общее количество статей по всему датасету: {total_articles_overall}")


Общее количество статей по всему датасету: 1191


In [ ]:
import pandas as pd
file_path = Path("data/dataset_241.csv")

try:

    df_241 = pd.read_csv(file_path, encoding='utf-8')
except pd.errors.ParserError:
    try:
    
        df_241 = pd.read_csv(file_path, encoding='utf-8', sep=';')
    except pd.errors.ParserError:
    
        df_241 = pd.read_csv(file_path, encoding='utf-8', header=None)

df_241.head(5)

,case_id,case_hash,region_id,region_name,case_type,instance,court_name,court_id,case_number,entry_date,...,end_date,considered_by,cui,is_text,meta_link,text_link,connected_links,db_add_date,db_update_date,text
0,906172292094,d2fc13dbfe,2,Республика Башкортостан,CRIMINAL,FIRST,Калининский районный суд г. Уфы Республики Баш...,03RS0002,1-30/2012 (1-534/2011;),2011-08-02,...,2013-03-26,NaN,03RS0002-01-2010-006738-75,False,http://kalininsky--bkr.sudrf.ru/modules.php?na...,NaN,NaN,2024-08-06,2024-08-06,NaN
1,806440353122,bbc3973d62,2,Республика Башкортостан,CRIMINAL,FIRST,Салаватский городской суд Республики Башкортостан,03RS0015,1-687/2012,2012-09-04,...,2012-10-29,Рассмотрено единолично судьей,03RS0015-01-2012-004198-54,True,http://salavatsky--bkr.sudrf.ru/modules.php?na...,http://salavatsky--bkr.sudrf.ru/modules.php?na...,NaN,2024-08-06,2024-08-06,ПРИГОВОР\nИменем Российской Федерации\nг.Салав...
2,742748742649,acef4663f9,86,Ханты-Мансийский автономный округ — Югра,CRIMINAL,FIRST,Сургутский городской суд Ханты-Мансийского авт...,86RS0004,1-1054/2012,2012-10-16,...,2012-11-09,NaN,NaN,False,http://surggor--hmao.sudrf.ru/modules.php?name...,NaN,NaN,2024-08-06,2024-08-06,NaN
3,800044188187,ba4659821b,41,Камчатский край,CRIMINAL,FIRST,Петропавловск-Камчатский городской суд Камчатс...,41RS0001,1-617/2012,2012-07-11,...,2012-12-11,NaN,NaN,False,http://p-kamchatsky--kam.sudrf.ru/modules.php?...,NaN,NaN,2024-08-06,2024-08-06,NaN
4,1040329320704,f23875a900,52,Нижегородская область,CRIMINAL,FIRST,Кстовский городской суд Нижегородской области,52RS0016,1-219/2012,2012-06-29,...,2012-10-16,Рассмотрено единолично судьей,52RS0016-01-2012-003253-59,False,http://kstovsky--nnov.sudrf.ru/modules.php?nam...,NaN,NaN,2024-08-06,2024-08-06,NaN


In [25]:
import pandas as pd
import re

df_241['accused'] = df_241['accused'].str.replace(r'\s+', ' ', regex=True).str.strip()
df_241['accused'] = df_241['accused'].str.replace(r'[–—]', '-', regex=True)

# Регулярное выражение для статьи УК РФ
article_pattern = r'(ст\.?\s*241(?:\s*ч\.?\s*[1-3])?)'

# Функция для извлечения уникальных статей в блоках с "<Информация скрыта>"
def extract_hidden_articles(text):
    results = []
    blocks = re.split(r'(<Информация скрыта>)', text)
    
    for i in range(len(blocks)):
        if blocks[i] == "<Информация скрыта>":
            if i + 1 < len(blocks):
                articles = set(re.findall(article_pattern, blocks[i + 1], flags=re.IGNORECASE))
                results.append(len(articles))
    
    return sum(results)

# Функция для извлечения уникальных статей по каждому доступному ФИО
def extract_fio_with_articles(text):
    fio_pattern = (
        r'([А-ЯЁA-Z]+(?:\s[А-ЯЁA-Z]+){2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z][а-яёa-z]+){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s[А-ЯЁA-Z]\.){1,2}'
        r'|[А-ЯЁA-Z][а-яёa-z]+(?:\s(?:уулу|оглы|кызы))\s[А-ЯЁA-Z][а-яёa-z]+'
        r'|<Информация скрыта>)'
    )
    fio_matches = re.findall(fio_pattern, text)
    results = []
    start_index = 0

    for i, fio in enumerate(fio_matches):
        fio = fio.strip()
        if fio == '<Информация скрыта>':
            continue

        fio_index = text.find(fio, start_index)
        if i + 1 < len(fio_matches):
            next_fio = fio_matches[i + 1]
            next_fio_index = text.find(next_fio, fio_index)
            text_after_fio = text[fio_index + len(fio): next_fio_index]
        else:
            text_after_fio = text[fio_index + len(fio):]
        
        articles = set(re.findall(article_pattern, text_after_fio, flags=re.IGNORECASE))
        results.append(len(articles))
        start_index = fio_index + len(fio)
    
    return sum(results)

# Применяем функции
df_241['hidden_articles_count'] = df_241['accused'].apply(extract_hidden_articles)
df_241['unique_articles_count'] = df_241['accused'].apply(extract_fio_with_articles)

# Считаем сумму двух колонок для доступных ФИО и "<Информация скрыта>"
df_241['total_articles_count'] = df_241['hidden_articles_count'] + df_241['unique_articles_count']

df_241[['case_number', 'hidden_articles_count', 'unique_articles_count', 'total_articles_count']].head(20)

total_articles_overall = df_241['total_articles_count'].sum()
print(f"Общее количество статей по всему датасету: {total_articles_overall}")


Общее количество статей по всему датасету: 3862
